In [1]:
import pandas as pd
import numpy as np
import eikon as ek
import sys
import config
ek.set_app_key(config.eikon_key)
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from scipy.stats.mstats import winsorize

In [2]:
hold = ek.get_data('SPY', fields = [ek.TR_Field('TR.ETPConstituentRIC',params={'SDate':'2020-11-17'})])[0]
hold = hold[hold['Constituent RIC'] != 'GOOG.OQ']
rics = [x for x in hold['Constituent RIC']]
if 'GOOG.OQ' in rics == True:
    rics.remove('GOOG.OQ')
    

In [3]:
valuation_fields = [ek.TR_Field('TR.GICSSector'),ek.TR_Field('TR.NetProfitMean', params={'Period':'FY2'}), ek.TR_Field('TR.EBITDAReportedMean',  params={'Period':'FY2'}),
                    ek.TR_Field('TR.EBITMean', params = {'Period':'FY2'}), ek.TR_Field('TR.TotalEquity', params = {'Period':'FY0'}),
                   ek.TR_Field('TR.CompanyMarketCap'), ek.TR_Field('TR.EV')]


valuation_dat = ek.get_data(rics, fields=valuation_fields)[0]

In [4]:
valuation_dat['Earnings_Yield'] = valuation_dat['Net Income - Mean']/valuation_dat['Company Market Cap']
valuation_dat['Book_Market'] = valuation_dat['Total Equity']/valuation_dat['Company Market Cap']
valuation_dat['EBITDA_EV'] = valuation_dat['EBITDA Reported - Mean']/valuation_dat['Enterprise Value (Daily Time Series)']
valuation_dat['EBIT_EV'] = valuation_dat['EBIT - Mean']/valuation_dat['Enterprise Value (Daily Time Series)']
valuation_dat_mkt = valuation_dat.filter(['Instrument', 'GICS Sector Name', 'Earnings_Yield', 'Book_Market', 'EBITDA_EV', 'EBIT_EV'])
min_max = RobustScaler()
valuation_dat_mkt[['Earnings_Yield', 'Book_Market', 'EBITDA_EV', 'EBIT_EV']] = min_max.fit_transform(valuation_dat_mkt[['Earnings_Yield', 'Book_Market', 'EBITDA_EV', 'EBIT_EV']])


In [5]:
valuation_dat_mkt['Valuation Composite'] = valuation_dat_mkt.mean(axis = 1)
valuation_dat_mkt[['Valuation Composite']] = min_max.fit_transform(valuation_dat_mkt[['Valuation Composite']])

In [6]:
fundam_fields = [ek.TR_Field('TR.GICSSector'), ek.TR_Field('TR.ROICMean', params={'Period':'FY2'}), ek.TR_Field('TR.WACC'), ek.TR_Field('TR.ROEMean', params={'Period':'FY2'}),
                 ek.TR_Field('TR.WACCCostofEquity'), ek.TR_Field('TR.LTGMean'), ek.TR_Field('TR.TtlDebtToTtlCapitalPct', params={'Period':'FY0'})]
fundamental_dat = ek.get_data(rics, fields=fundam_fields)[0]

In [7]:
fundamental_dat

,Instrument,GICS Sector Name,Return On Invested Capital - Mean,"Weighted Average Cost of Capital, (%)",Return On Equity - Mean,"WACC Cost of Equity, (%)",Long Term Growth - Mean,"Total Debt to Total Capital, Percent"
0,SHW.N,Materials,18.04000,6.582432,51.52020,7.112172,9.57320,67.808096
1,DTE.N,Utilities,5.70000,3.377334,10.94983,4.523969,6.03333,59.590304
2,LOW.N,Consumer Discretionary,26.05000,7.943424,323.06000,9.117949,20.58450,90.732212
3,WBA.OQ,Consumer Staples,12.20000,3.549397,15.69071,4.182457,6.98900,44.277768
4,MAR.OQ,Consumer Discretionary,16.83333,9.488921,152.33000,12.222039,-11.93000,93.962037
...,...,...,...,...,...,...,...,...
500,CI.N,Health Care,7.80000,4.354232,14.93488,6.247277,10.24925,45.185176
501,FIS.N,Information Technology,7.10000,4.955753,7.33717,6.009338,12.24733,28.991500
502,FTNT.OQ,Information Technology,16.50000,7.565880,49.09263,7.565880,20.30275,0.000000
503,TXN.OQ,Information Technology,44.46667,7.070344,64.44213,7.360454,10.00000,39.449354


In [8]:
fundamental_1 = fundamental_dat[fundamental_dat['Return On Invested Capital - Mean'].isnull()]
fundamental_1['Spread'] = fundamental_1['Return On Equity - Mean'] - fundamental_1['WACC Cost of Equity, (%)']
fundamental_2 = fundamental_dat[fundamental_dat['Return On Invested Capital - Mean'].notnull()]
fundamental_2['Spread'] = fundamental_2['Return On Invested Capital - Mean'] - fundamental_2['WACC Cost of Equity, (%)']
fundamental_score = pd.concat([fundamental_2, fundamental_1])
fundamental_score['Equity To Total Cap'] = 100 - fundamental_score['Total Debt to Total Capital, Percent']
fundamental_score[['Spread', 'Long Term Growth - Mean', 'Equity To Total Cap']] = min_max.fit_transform(fundamental_score[['Spread', 'Long Term Growth - Mean', 'Equity To Total Cap']])
fundamental_score['Fund_score'] = fundamental_score[['Spread', 'Long Term Growth - Mean', 'Equity To Total Cap']].sum(axis = 1)
fundamental_score['Fund_score'] = min_max.fit_transform(fundamental_score[['Fund_score']])

C:\Users\f16se\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\f16se\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
mom_fields = [ek.TR_Field('TR.GICSSector'), ek.TR_Field('TR.PricePctChg3M'), ek.TR_Field('TR.PricePctChg6M'), ek.TR_Field('TR.PricePctChg1Y'), ek.TR_Field('TR.BetaWklyAdj2Y')]
momentum = ek.get_data(rics, fields=mom_fields)[0]
spy_rets = ek.get_data('SPY', fields=mom_fields)[0]

In [10]:
mom_1 = momentum[momentum['Weekly Beta, Adjusted - 2 Year'].notnull()]
mom_2 = momentum[momentum['Weekly Beta, Adjusted - 2 Year'].isnull()]

In [11]:

mom_1['3_month'] = mom_1['3-month Price PCT Change'] - (spy_rets['3-month Price PCT Change'].iloc[0] * mom_1['Weekly Beta, Adjusted - 2 Year'])
mom_1['6_month'] = mom_1['6-month Price PCT Change'] - (spy_rets['6-month Price PCT Change'].iloc[0] * mom_1['Weekly Beta, Adjusted - 2 Year'])
mom_1['12_month'] = mom_1['1-year Price PCT Change'] - (spy_rets['1-year Price PCT Change'].iloc[0] * mom_1['Weekly Beta, Adjusted - 2 Year'])
mom_2['3_month'] = mom_2['3-month Price PCT Change'] - spy_rets['3-month Price PCT Change'].iloc[0]
mom_2['6_month'] = mom_2['6-month Price PCT Change'] - spy_rets['6-month Price PCT Change'].iloc[0]
mom_2['12_month'] = mom_2['1-year Price PCT Change'] - spy_rets['1-year Price PCT Change'].iloc[0]

momentum = pd.concat([mom_1, mom_2])


C:\Users\f16se\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\f16se\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\f16se\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [12]:
momentum = momentum.filter(['Instrument', 'GICS Sector Name', '3_month', '6_month', '12_month'])

In [13]:
momentum[['3_month', '6_month', '12_month']] = min_max.fit_transform(momentum[['3_month', '6_month', '12_month']])
momentum['Momentum Score'] = momentum.sum(axis = 1)
momentum[['Momentum Score']] = min_max.fit_transform(momentum[['Momentum Score']])

In [14]:
score_tab = valuation_dat_mkt.filter(['Instrument', 'GICS Sector Name','Valuation Composite'])
score_tab = score_tab.join(momentum['Momentum Score'])
score_tab = score_tab.join(fundamental_score['Fund_score'])


In [15]:
names = ek.get_data(rics, fields='TR.CommonName')[0]

In [16]:
score_tab = score_tab.join(names['Company Common Name'])

In [17]:
score_tab = score_tab.filter(['Instrument', 'Company Common Name', 'GICS Sector Name', 'Valuation Composite', 'Momentum Score', 'Fund_score'])
score_tab = score_tab.replace(np.nan, 0)
score_tab['Short Term Score'] = score_tab['Momentum Score']*0.5 + 0.3*score_tab['Fund_score'] + 0.2*score_tab['Valuation Composite']
score_tab['Long Term Score'] = score_tab['Momentum Score']*0.2 + 0.3*score_tab['Fund_score'] + 0.5*score_tab['Valuation Composite']
score_tab[['Short Term Score', 'Long Term Score']] = min_max.fit_transform(score_tab[['Short Term Score', 'Long Term Score']])
rescale = MinMaxScaler()

score_tab[['Valuation Composite', 'Momentum Score', 'Fund_score', 'Short Term Score', 'Long Term Score']] = round(score_tab[['Valuation Composite', 'Momentum Score', 'Fund_score', 'Short Term Score', 'Long Term Score']]*100,0)
score_tab = score_tab.rename(columns = {'Valuation Composite': 'Valuation Score', 'Fund_score': 'Fundamental Score', 'GICS Sector Name': 'Sector', 'Company Common Name': 'Name'})




In [18]:
tickers = []
for r in rics:
    x = r.split('.',1)
    tickers.append(x[0])
score_tab['Instrument'] = tickers
score_tab = score_tab.rename(columns = {'Instrument':'Symbol'})

In [19]:
import gspread

gc = gspread.oauth()

sh = gc.open("score spreadsheet")
worksheet = sh.worksheet(title="A worksheet")
worksheet.update([score_tab.columns.values.tolist()]+score_tab.values.tolist())



{'spreadsheetId': '1yEncOOYS18ZWXpDz_UY3Xx2xn-7zLNvq0KgAIVMXmdk',
 'updatedRange': "'A worksheet'!A1:H506",
 'updatedRows': 506,
 'updatedColumns': 8,
 'updatedCells': 4048}